# Step Build Base Data

In [21]:
# Set start time

from datetime import datetime
start_time = datetime.now()

In [22]:
# Import the data
from tensorflow.keras import layers, models, Model
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

df = pd.read_csv('./Resources/CO_Accidents_March23.csv')
df.head()

,Unnamed: 0,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,A-512446,Source2,3,2022-09-08 13:03:18,2022-09-08 13:32:26,39.848122,-104.985306,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,A-512565,Source2,1,2022-09-08 08:44:08,2022-09-08 09:13:40,39.776600,-105.062630,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,A-512637,Source2,2,2022-09-08 06:57:38,2022-09-08 07:27:23,39.899731,-104.868507,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,A-512643,Source2,1,2022-09-08 08:45:51,2022-09-08 09:15:28,39.618809,-104.773201,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,A-512644,Source2,3,2022-09-08 08:42:23,2022-09-08 09:11:57,39.725094,-105.012817,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day


In [23]:
  # extract relevant columns

relevant_df = df[['County', 'Severity', 'Start_Time', 'Street',
                  'Weather_Condition']]

relevant_df['Start_Time'] = pd.to_datetime(relevant_df['Start_Time'])
# on colab and mac use ,format='mixed')

relevant_df['Start_Time_int'] = relevant_df['Start_Time'].dt.strftime('%Y%m%d').astype(int)

[print('accident_count = ', relevant_df.count())]


ValueError: time data '2022-09-08 13:03:18' does not match format 'mixed' (match)

In [ ]:
# Create DataFrame that containes only Colorado

CO_crashes_df =  relevant_df.loc[(df["State"] == "CO")]
CO_crashes_df['Severity'].value_counts()

2    56105
3    26306
4     7504
1      970
Name: Severity, dtype: int64

In [ ]:
# Update CO_crashes_df with numeric county code

county_dict = {
"Adams":1,"Alamosa":3,"Arapahoe":5,"Archuleta":7,"Baca":9,"Bent":11,"Boulder":13,"Broomfield":14,"Chaffee":15,"Cheyenne":17,"Clear Creek":19,"Conejos":21,"Costilla":23,"Crowley":25,"Custer":27,
"Delta":29,"Denver":31,"Dolores":33,"Douglas":35,"Eagle":37,"El Paso":41,"Elbert":39,"Fremont":43,"Garfield":45,"Gilpin":47,"Grand":49,"Gunnison":51,"Hinsdale":53,"Huerfano":55,
"Jackson":57,"Jefferson":59,"Kiowa":61,"Kit Carson":63,"La Plata":67,"Lake":65,"Larimer":69,"Las Animas":71,"Lincoln":73,"Logan":75,"Mesa":77,"Mineral":79,
"Moffat":81,"Montezuma":83,"Montrose":85,"Morgan":87,"Otero":89,"Ouray":91,"Park":93,"Phillips":95,"Pitkin":97,"Prowers":99,"Pueblo":101,"Rio Blanco":103,"Rio Grande":105,"Routt":107,
"Saguache":109,"San Juan":111,"San Miguel":113,"Sedgwick":115,"Summit":117,"Teller":119,"Washington":121,"Weld":123,"Yuma":125
}

CO_crashes_df_updated =  CO_crashes_df.replace({"County": county_dict})

CO_crashes_df_updated.head(5)


,County,Severity,Start_Time,Street,Weather_Condition,Start_Time_int
0,1,3,2022-09-08 13:03:18,I-25 N,Partly Cloudy,20220908
1,59,1,2022-09-08 08:44:08,W 44th Ave,Fair,20220908
2,1,2,2022-09-08 06:57:38,E 112th Ave,Fair,20220908
3,5,1,2022-09-08 08:45:51,S Tower Rd,Fair,20220908
4,31,3,2022-09-08 08:42:23,I-25,Fair,20220908


In [ ]:
# Update Weather_Conditions and combine duplicates
CO_crashes_df_updated['Weather_Condition'] = CO_crashes_df_updated['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','_',str(x)))

CO_crashes_df_updated['Weather_Condition'] = CO_crashes_df_updated['Weather_Condition'].replace({'Overcast': 'Cloudy', 
                                                                'Light Freezing Drizzle': 'Light Drizzle', 
                                                                'Snow _ Windy':'Light Snow _ Windy',
                                                                'Light Rain _ Windy': 'Light Snow Windy',
                                                                'N_A Precipitation': 'Rain',
                                                                'Heavy Snow _ Windy': 'Snow',
                                                                'Haze _ Windy': 'Haze',
                                                                'Light Rain Shower': 'Rain',
                                                                'Fog _ Windy': 'Fog',
                                                                'T_Storm _ Windy': 'T_Storm',
                                                                'Patches of Fog': 'Fog',
                                                                'Heavy Rain': 'Rain',
                                                                'Thunder _ Windy': 'Thunder',
                                                                'Blowing Dust _ Windy': 'Cloudy Windy',
                                                                'Blowing Snow _ Windy': 'Mostly Cloudy _ Windy',
                                                                'Heavy T_Storm': 'T_Storm',
                                                                'Light Freezing Fog': 'Fog',
                                                                'Heavy T_Storm _ Windy': 'T_Storm',
                                                                'Partial Fog': 'Fog',
                                                                'Widespread Dust _ Windy': 'Fog',
                                                                'Scattered Clouds': 'Cloudy',
                                                                'Drizzle and Fog': 'Fog',
                                                                'Rain _ Windy': 'Windy',
                                                                'Squalls _ Windy': 'Windy',
                                                                'Blowing Snow': 'Snow',
                                                                'Smoke _ Windy': 'Smoke',
                                                                'Light Snow with Thunder': 'Light Snow',
                                                                'Heavy Rain _ Windy': 'Rain',
                                                                'Light Freezing Rain': 'Rain',
                                                                'Clear': 'Fair',
                                                                'Blowing Dust': 'Mostly Cloudy _ Windy',
                                                                'Light Rain Shower _ Windy': 'Light Snow  Windy',
                                                                'Heavy Drizzle': 'Light Drizzle',
                                                                'Thunder _ Wintry Mix': 'Wintry Mix',
                                                                'Light Drizzle _ Windy': 'Light Drizzle',
                                                                'Light Snow Shower': 'Light Snow',
                                                                'Light Thunderstorms and Snow': 'Light Snow',
                                                                'Light Thunderstorms and Rain': 'Light Rain',
                                                                'Freezing Drizzle': 'Drizzle',
                                                                'Squalls': 'Fair _ Windy',
                                                                'Windy': 'Fair _ Windy',
                                                                'Thunderstorm': 'Thunder',
                                                                'Coloudy Wind': 'Mostly Cloudy _ Windy',
                                                                'Drizzle': 'Light Drizzle',
                                                                'Cloudy Windy': 'Mostly Cloudy _ Windy',
                                                                'Clouds': 'Cloudy',
                                                                'Light Snow  Windy': 'Light Snow _ Windy',
                                                                'Thunderstorms and Rain': 'T_Storm',
                                                                'Light Thunderstorm': 'T_Storm',
                                                                'Snow and Thunder': 'Thunder',
                                                                'Mist': 'Cloudy',
                                                                'Heavy Thunderstorms and Rain': 'Light Rain with Thunder',
                                                                'Funnel Cloud': 'Cloudy',
                                                                'Heavy Blowing Snow': 'Snow'
                                                    })

pd.options.display.max_rows=100
CO_crashes_df_updated['Weather_Condition'].value_counts()

Fair                       32024
Mostly Cloudy              16447
Partly Cloudy              13865
Cloudy                     12382
Light Snow                  6424
nan                         1410
Snow                        1145
Fair _ Windy                1064
Light Rain                  1024
Fog                          748
Mostly Cloudy _ Windy        574
Partly Cloudy _ Windy        473
Haze                         430
Light Drizzle                404
Heavy Snow                   382
Rain                         276
Light Snow _ Windy           242
Cloudy _ Windy               234
Thunder in the Vicinity      217
Showers in the Vicinity      205
Smoke                        202
Light Rain with Thunder      166
Thunder                      156
T_Storm                      156
Wintry Mix                   154
Light Snow Windy              48
Cloudy Windy                  16
Windy                         12
Light Snow  Windy              2
Light Snow Showers             2
Drizzle   

In [ ]:
# Encode ID

le = LabelEncoder()
CO_crashes_df_updated['ID'] = le.fit_transform(df['ID'])
CO_crashes_df_updated.set_index('ID')

CO_crashes_df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90885 entries, 0 to 90884
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   County             90885 non-null  int64         
 1   Severity           90885 non-null  int64         
 2   Start_Time         90885 non-null  datetime64[ns]
 3   Street             90727 non-null  object        
 4   Weather_Condition  90885 non-null  object        
 5   Start_Time_int     90885 non-null  int32         
 6   ID                 90885 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int64(2), object(2)
memory usage: 4.9+ MB


In [ ]:
# Show counts 

CO_crashes_df_updated.count()

County               90885
Severity             90885
Start_Time           90885
Street               90727
Weather_Condition    90885
Start_Time_int       90885
ID                   90885
dtype: int64

In [ ]:
# Write data to Resources/result_files/step1_build_base_data.csv file
CO_crashes_df_updated.to_csv('./Resources/result_files/step1_build_base_data.csv', sep=',', encoding='utf-8')

In [ ]:

# Calculate Duration
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


Duration: 0:00:01.832423
